# Tutorial B5 (Automated CPA Attack With Analyzer)

This tutorial will take you through a complete attack on a software AES implementation. The specific implementation being attacked is a well-known AES implementation written in C, which is likely to be similar to other implementations used by proprietary systems.

## Capturing Power Traces

### Setup

We'll use some helper scripts to make setup and programming easier. If you're using an XMEGA or STM (CWLITEARM) target, binaries with the correct should be setup for you:

In [1]:
%run "Helper Scripts/CWLite_Connect.ipynb"

In [2]:
%run "Helper Scripts/Setup_Target_Generic.ipynb"

In [3]:
# uncomment based on your target
#%run "Helper Scripts/Program_XMEGA.ipynb"
%run "Helper Scripts/Program_STM.ipynb"
#%run "Helper Scripts/No_Programmer.ipynb"
fw_path = "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-cwlitearm.hex"

In [4]:
# program the target
program_target(scope, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 6035 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 6035 bytes


In addition, before we capture our traces, we'll need to create a ChipWhipserer project, since that's what Analyzer expects for an input:

In [ ]:
from chipwhisperer.common.api.ProjectFormat import ProjectFormat
project = ProjectFormat()
project.setFilename("jupyter_test")

And we can get the class used to hold our traces by:

In [ ]:
tc = project.getTraceFormat()

### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and our new trace into our trace class. We'll also keep track of our keys manually for checking our answer later.

In [5]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time

ktp = AcqKeyTextPattern_Basic(target=target)

keys = []
N = 50  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    #traces.append(scope.getLastTrace())
    tc.addTrace(scope.getLastTrace(), text, "", key)

Capturing traces: 100%|██████████| 50/50 [00:08<00:00,  5.72it/s]


Now that we have our traces, we need to tell the project that the traces are loaded and add them to the project's trace manager.

In [ ]:
tc._isloaded = True
project.traceManager().appendSegment(tc)

We're now done with the ChipWhisperer hardware, so we should disconnect from the scope and target:

In [6]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Analysis

Now that we have our traces, we can begin our attack! We'll start off by importing everything we need for the attack:

In [ ]:
from chipwhisperer.analyzer.attacks.cpa import CPA
from chipwhisperer.analyzer.attacks.cpa_algorithms.progressive import CPAProgressive
from chipwhisperer.analyzer.attacks.models.AES128_8bit import AES128_8bit, SBox_output
from chipwhisperer.analyzer.preprocessing.add_noise_random import AddNoiseRandom
from chipwhisperer.common.results.outputvstime import OutputVsTimeNoGUI

Next, we'll add our traces to a preprocessing module. This isn't necessary (you can feed `project.traceManager` right into `attack.setTraceSource()`, but this shows how to use preprocessing modules:

In [ ]:
ppmod = AddNoiseRandom(project.traceManager())
ppmod.noise = 0.05
ppmod.enabled = False

And then we can setup our attack:

In [8]:
attack = CPA()
N = 50 #number of traces

leak_model = AES128_8bit(SBox_output)
attack.setAnalysisAlgorithm(CPAProgressive, leak_model)
attack.setTraceSource(ppmod)
attack.setTraceStart(0)
attack.setTracesPerAttack(N)
attack.setIterations(1)
attack.setReportingInterval(10)
attack.setTargetSubkeys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
attack.setPointRange((0, -1))

And then actually run it:

In [ ]:
stats = attack.processTracesNoGUI()

Once you see the above block complete, all the heavylifting is done! All that's left is to actually look at the data. Everything important is contained in the `stats` object that `attack.processTracesNoGUI()` returned.

We can find the max correlation for every one of the subkey by calling `stats.findMaximums()`, which returns a list of the subkeys, the PGE (more on that in Tutorial B6), and the correlation (which is a value between 0 and 1 that effectively tells us how well our guess fit the data). 

It's up to us to print and interpret the data. Below, you can find a simple loop to print out our best guess for the key, the actual key, and the correlation. With any luck, you should see a bunch of numbers printed which match the ones in parentheses and some text at the bottom that says we got the encryption key.

In [20]:
key = keys[0]
i = 0
all_right = True
for bnum in stats.findMaximums():
    print("Best Guess (Actual) = 0x{:02X} (0x{:02X}), Corr = {}".format(bnum[0][0], key[i], bnum[0][2]))
    if bnum[0][0] != key[i]:
        all_right = False
    i += 1
    
if all_right:
    print("We guessed the encryption key!")
else:
    print("We didn't guess the encryption key")

Best Guess (Actual) = 0x2B (0x2B), Corr = 0.9310820685784111
Best Guess (Actual) = 0x7E (0x7E), Corr = 0.9039113526898941
Best Guess (Actual) = 0x15 (0x15), Corr = 0.923218898956149
Best Guess (Actual) = 0x16 (0x16), Corr = 0.865107227127149
Best Guess (Actual) = 0x28 (0x28), Corr = 0.9177310619915472
Best Guess (Actual) = 0xAE (0xAE), Corr = 0.9133718421265529
Best Guess (Actual) = 0xD2 (0xD2), Corr = 0.9085171285715561
Best Guess (Actual) = 0xA6 (0xA6), Corr = 0.8766696063600293
Best Guess (Actual) = 0xAB (0xAB), Corr = 0.9061065479957335
Best Guess (Actual) = 0xF7 (0xF7), Corr = 0.7963219879594106
Best Guess (Actual) = 0x15 (0x15), Corr = 0.9247435968336774
Best Guess (Actual) = 0x88 (0x88), Corr = 0.91432693762354
Best Guess (Actual) = 0x09 (0x09), Corr = 0.9095535310951552
Best Guess (Actual) = 0xCF (0xCF), Corr = 0.888547570218893
Best Guess (Actual) = 0x4F (0x4F), Corr = 0.8709750629075454
Best Guess (Actual) = 0x3C (0x3C), Corr = 0.8992904483642833
We guessed the encryption key

That's all well and good, but how good much better was our guess than the other ones? To show that, we can use ChipWhisperer's `OutputVsTimeNoGUI`:

In [27]:
outvstime = OutputVsTimeNoGUI(stats, key)
ret = outvstime.getPlotData(15)
xrange = ret[0]

And then plot the data:

In [28]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()
p.line(xrange, ret[2], line_color='green')
p.line(xrange, ret[3], line_color='green')

p.line(xrange, ret[1], line_color='red')
show(p)

Loading BokehJS ...

You should see a graph of red and green in time (samples). In red is the correlation of the correct subkey for the first byte, while the rest are in green.

You should see two or three distinctive red spikes. The first is the spot where the sbox lookup for the subkey we guessed actually happens (the later ones are from later steps in the AES operation).

What about the rest of the bytes in the key? We can get and plot that easily as well:

In [19]:
rets = []
for i in range(0, 16):
    rets.append(outvstime.getPlotData(i))

p = figure()
for ret in rets:
    p.line(xrange, ret[2], line_color='green')
    p.line(xrange, ret[3], line_color='green')
    
for ret in rets:
    p.line(xrange, ret[1], line_color='red')

show(p)

## Conclusion

You should now have completed a successful CPA attack! 

You can move onto more advanced tutorials, especially showing you how the actual attack works when performed manually (Tutorial B6). This tutorial also utilized tiny-AES128-C for Arm targets, which uses the same operations as the XMEGA target. A later tutorial will preform this attack on a more typical 32 bit AES implementation.